# Borehole case study with QGIS application

Looking at boreholes done since 01-01-2019:

In [1]:
import pydov
from pydov.search.boring import BoringSearch
boring = BoringSearch()

# all boreholes done since 01-01-2019
from owslib.fes import PropertyIsGreaterThanOrEqualTo
query = PropertyIsGreaterThanOrEqualTo(propertyname='datum_aanvang', literal='2019-01-01')
df = boring.search(query=query,
                   return_fields=('id', 'pkey_boring', 'diepte_boring_tot', 'boornummer', 'uitvoerder',
                   'datum_aanvang', 'doel'))
df.head()

,id,pkey_boring,diepte_boring_tot,boornummer,uitvoerder,datum_aanvang,doel
0,5013936,https://www.dov.vlaanderen.be/data/boring/2018...,75.0,1401-B190001,NRGeo,2019-01-07,Geothermie
1,5014548,https://www.dov.vlaanderen.be/data/boring/2018...,6.0,1429-B-W18144,A. Vyncke NV,2019-02-05,Drainage en bemaling
2,5018040,https://www.dov.vlaanderen.be/data/boring/2019...,120.0,1405-B419196,BACKX,2019-01-18,Geothermie
3,5017608,https://www.dov.vlaanderen.be/data/boring/2019...,8.0,1607-B2019001,Vannieuwenborg Alexander,2019-01-05,Grondwaterwinning
4,5017609,https://www.dov.vlaanderen.be/data/boring/2019...,8.0,1607-B2019002,Vannieuwenborg Alexander,2019-01-05,Grondwaterwinning


With exporting the DataFrame to excel the user can easily analyse this data however they choose. An example of how DataFrames can be grouped is also added below:

In [2]:
df_sorted = df.groupby(['uitvoerder','doel', 'diepte_boring_tot']).count().pkey_boring     # counts the amount of boreholes (unique pkey_boring urls) per depth
df_sorted.head()

# save to excel 
# df.to_excel('Boreholes_Flanders.xlsx')
# df_sorted.to_excel('Boreholes_Flanders_sorted.xlsx')

uitvoerder    doel                  diepte_boring_tot
A. Vyncke NV  Drainage en bemaling  5.0                  3
                                    6.0                  7
                                    7.0                  4
                                    8.0                  1
                                    9.0                  1
Name: pkey_boring, dtype: int64

## Equal depths? 
When the data was inspected, the author of this case study noticed some inconsistencies in the borehole data. 
The question was raised if the borehole depth was equal to the borehole interpretation, as there were a few cases where the borehole depth was 0 meters, but there was an interpretation that showed a depth of e.g. 22 meters (in this case the interpretation was of the coded lithology 'gecodeerde lithologie'). This was checked for the coded lithology via: 

In [3]:
from pydov.search.boring import BoringSearch
from pydov.search.interpretaties import GecodeerdeLithologieSearch, LithologischeBeschrijvingenSearch
boring = BoringSearch()
gl = GecodeerdeLithologieSearch()
lb = LithologischeBeschrijvingenSearch()

# all boreholes since 01-01-2019
from owslib.fes import PropertyIsGreaterThanOrEqualTo
# from pydov.util.query import Join => broken at the time
query = PropertyIsGreaterThanOrEqualTo(propertyname='datum_aanvang', literal='2019-01-01')
boringen = boring.search(query=query, return_fields=('pkey_boring', 'uitvoerder', 'doel', 'x', 'y', 'diepte_boring_tot'))

gec_lith = gl.search(query=PropertyIsGreaterThanOrEqualTo(propertyname='Datum', literal='2019-01-01'),
                     return_fields=('pkey_boring', 'diepte_tot_m', 'x', 'y'))

df = boringen.merge(gec_lith)       # only intersecting data is merged (so if no 'gecodeerde lithologische beschrijving' data is ignored)

df['equal_depths'] = df.diepte_boring_tot >= df.diepte_tot_m  # checks if depths are larger than or the same in an added column 
df.head()

,pkey_boring,uitvoerder,doel,x,y,diepte_boring_tot,diepte_tot_m,equal_depths
0,https://www.dov.vlaanderen.be/data/boring/2019...,GEOLAB,Geotechnisch onderzoek,207214.6,166962.1,20.0,20.0,True
1,https://www.dov.vlaanderen.be/data/boring/2019...,GEOLAB,Geotechnisch onderzoek,207298.7,167028.5,15.0,15.0,True
2,https://www.dov.vlaanderen.be/data/boring/2019...,GEOLAB,Geotechnisch onderzoek,49072.6,214684.7,7.0,7.0,True
3,https://www.dov.vlaanderen.be/data/boring/2019...,GEOLAB,Geotechnisch onderzoek,70708.9,208860.4,25.0,25.0,True
4,https://www.dov.vlaanderen.be/data/boring/2019...,GEOLAB,Geotechnisch onderzoek,152023.7,210194.1,18.5,18.5,True


And the lithological interpretation: 

In [4]:
lith = lb.search(query=PropertyIsGreaterThanOrEqualTo(propertyname='Datum', literal='2019-01-01'),
                     return_fields=('pkey_boring', 'diepte_tot_m', 'x', 'y'))

df2 = boringen.merge(lith)       # only intersecting data is merged (so if no 'lithologische beschrijving', data is not taken up)
df2['equal_depths'] = df2.diepte_boring_tot >= df2.diepte_tot_m  # checks if depths are larger than or the same as interpretation depth 
df2.head()

,pkey_boring,uitvoerder,doel,x,y,diepte_boring_tot,diepte_tot_m,equal_depths
0,https://www.dov.vlaanderen.be/data/boring/2019...,Vannieuwenborg Alexander,Grondwaterwinning,29950.18,200699.08,8.0,8.0,True
1,https://www.dov.vlaanderen.be/data/boring/2019...,Vannieuwenborg Alexander,Grondwaterwinning,31835.04,201931.49,8.0,8.0,True
2,https://www.dov.vlaanderen.be/data/boring/2019...,BOSTOEN JOHAN,Drainage en bemaling,29620.00,200775.00,6.0,6.0,True
3,https://www.dov.vlaanderen.be/data/boring/2019...,BOSTOEN JOHAN,Drainage en bemaling,56996.00,164465.00,5.0,5.0,True
4,https://www.dov.vlaanderen.be/data/boring/2019...,Putboringen Van Deynse,Drainage en bemaling,50971.49,183331.98,5.0,5.0,True


## QGIS application
In Flanders there are certain depthcriteria concerning vertical boreholes and boreholes for groundwaterextraction. These depthcriteria determine whether a report has to be made and/or a permit is needed to drill at the chosen location. 
In this case we are interested in comparing the depthcriteria 53.8, concerning groundwater extraction, to the depth of boreholes with a purpose ('doel') of groundwater extraction.  


In [5]:
from pydov.search.boring import BoringSearch
boring = BoringSearch()

# all boreholes since 01-01-2019 with purpose = groundwater extraction
from owslib.fes import PropertyIsGreaterThanOrEqualTo, And, PropertyIsEqualTo

query = And([PropertyIsGreaterThanOrEqualTo(propertyname='datum_aanvang', literal='2019-01-01'),
             PropertyIsEqualTo(propertyname='doel', literal='Grondwaterwinning')])

df = boring.search(query=query, return_fields=('pkey_boring', 'uitvoerder', 'doel', 'diepte_boring_tot', 'x', 'y'))
df.head()

# df.to_csv('Boreholes_2019_groundwater_extraction.csv')

,pkey_boring,uitvoerder,doel,diepte_boring_tot,x,y
0,https://www.dov.vlaanderen.be/data/boring/2019...,Vannieuwenborg Alexander,Grondwaterwinning,8.0,29950.18,200699.08
1,https://www.dov.vlaanderen.be/data/boring/2019...,Vannieuwenborg Alexander,Grondwaterwinning,8.0,31835.04,201931.49
2,https://www.dov.vlaanderen.be/data/boring/2019...,VANHECKE,Grondwaterwinning,46.0,78092.39,187983.10
3,https://www.dov.vlaanderen.be/data/boring/2019...,VANHECKE,Grondwaterwinning,15.0,76167.12,171596.80
4,https://www.dov.vlaanderen.be/data/boring/2019...,Putboringen Van Deynse,Grondwaterwinning,9.0,88382.11,210651.42


With this csv file and the x and y coördinates in it, the boreholes can be added to QGIS as points. The coördinate system is Lambert72 (EPSG:31370) (standard for DOV). 
The depthcriteria 53.8 map is available from DOV via WCS (https://www.dov.vlaanderen.be/geoserver/wcs?) and can easily be imported in QGIS. 
<img src="Images/QGIS_boreholes-Copy1.PNG" /> 
The 'point sampling tool' was used to add the depthcriteria at the location of the borehole to the borehole data, and exported back to a csv file.

The user can now analyse the data to their heart's content in their program of choice (e.g. Excel).